# Search for  GEMS SL in the objects Tables  ECDFS in LSSTComCam

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- member : DESC, rubin-inkind
- creation date : 2025-05-25
- last update : 2025-05-27: remove selection on parentObjectId=0 for the SL
- last update 2025-09-10 : setting up for the RSP (from usdf)

**LSST data products:** `deep_coadd`, `CcdVisit`, `Visit`, `Object`

**Packages:** `lsst.rsp`, `lsst.daf.butler`, `lsst.afw.display`

In [ ]:
FLAG_RSPDP1 = True

### Note on work done on LSSTComCam Commissioning : https://sitcomtn-149.lsst.io/
### Note Data Product definition Document : https://lse-163.lsst.io/

In [ ]:
import sys
import matplotlib.pyplot as plt
import lsst.afw.display as afwDisplay
import numpy as np
import pandas as pd
from astropy.time import Time
from astropy.io import fits
# %matplotlib widget
import copy  

In [ ]:
from lsst.geom import Point2D, Point2I, Box2I, Extent2I
from lsst.afw.image import ExposureF

In [ ]:
afwDisplay.setDefaultBackend("firefly")

In [ ]:
import lsst.geom as geom
from lsst.geom import SpherePoint, degrees, Point2D, Point2I, Extent2I
from lsst.skymap import PatchInfo, Index2D

In [ ]:
# For Angle conversion
from astropy.coordinates import Angle
import astropy.units as u
from astropy.coordinates import SkyCoord

In [ ]:
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.labelsize"] = "x-large"
plt.rcParams["axes.titlesize"] = "x-large"
plt.rcParams["xtick.labelsize"] = "x-large"
plt.rcParams["ytick.labelsize"] = "x-large"

In [ ]:
import gc

In [ ]:
import traceback

In [ ]:
# Define butler
from lsst.daf.butler import Butler

In [ ]:
!eups list lsst_distrib

In [ ]:
def nJy_to_ab_mag(f_njy):
    """Convert scalar or array flux in nJy to AB magnitude."""
    f_njy = np.asarray(f_njy)
    mag = np.full_like(f_njy, fill_value=np.nan, dtype=float)
    mask = f_njy > 0
    mag[mask] = -2.5 * np.log10(f_njy[mask]) + 31.4
    return mag


def nJy_err_to_ab_err(f_njy, f_err):
    """Propagate flux error to magnitude error."""
    f_njy = np.asarray(f_njy)
    f_err = np.asarray(f_err)
    mag_err = np.full_like(f_njy, fill_value=np.nan, dtype=float)
    mask = (f_njy > 0) & (f_err > 0)
    mag_err[mask] = (2.5 / np.log(10)) * (f_err[mask] / f_njy[mask])
    return mag_err

In [ ]:
def ab_mag_to_nJy(mag_ab):
    """Convert AB magnitude to flux in nanojanskys."""
    return 10 ** ((31.4 - mag_ab) / 2.5)

In [ ]:
def extract_deepCoadd_cutout(
    ra_deg,
    dec_deg,
    tractInfo,
    patchInfo,
    band,
    butler,
    skymapName,
    cutout_size_pixels,
    collection,
):
    """
    Extract a square cutout from a deepCoadd image centered on given sky coordinates.

    Parameters
    ----------
    ra_deg : float
        Right ascension in degrees.
    dec_deg : float
        Declination in degrees.
    tractInfo : lsst.skymap.TractInfo
        Tract information object (from SkyMap).
    patchInfo : lsst.skymap.PatchInfo
        Patch information object (from SkyMap).
    band : str
        Photometric band (e.g., 'i', 'r', 'g').
    butler : lsst.daf.butler.Butler
        Butler instance to access LSST data.
    cutout_size_pixels : int, optional
        Size of the cutout in pixels (square region), default is 200.
    collection : str, optional
        Name of the collection containing deepCoadd data.

    Returns
    -------
    cutout : lsst.afw.image.ExposureF
        The extracted image cutout.
    wcs : lsst.afw.geom.SkyWcs
        World Coordinate System associated with the cutout.
    metadata : lsst.daf.base.PropertyList
        FITS metadata header for the cutout image.
    """

    # Build dataId for the deepCoadd image
    tract = tractInfo.getId()
    patch = patchInfo.getSequentialIndex()
    dataId = dict(tract=tract, patch=patch, band=band, skymap=skymapName)

    # Retrieve the deepCoadd exposure
    if FLAG_RSPDP1:
        exposure = butler.get("deep_coadd", dataId=dataId, collections=collection)
    else:
        exposure = butler.get("deepCoadd", dataId=dataId, collections=collection)
        
    image_bbox = exposure.getBBox()

    # Convert sky coordinates (RA, Dec) to pixel coordinates using WCS
    coord = SpherePoint(ra_deg, dec_deg, degrees)
    wcs = exposure.getWcs()
    pixel_center = wcs.skyToPixel(coord)

    half_size = cutout_size_pixels // 2

    # Centre du cutout
    center_x = int(pixel_center.getX())
    center_y = int(pixel_center.getY())

    # Coordonnées du coin en bas à gauche
    corner_x = max(center_x - half_size, image_bbox.getMinX())
    corner_y = max(center_y - half_size, image_bbox.getMinY())

    # Ne pas dépasser la taille max de l'image
    corner_x = min(corner_x, image_bbox.getMaxX() - cutout_size_pixels)
    corner_y = min(corner_y, image_bbox.getMaxY() - cutout_size_pixels)

    # Define a square bounding box around the target pixel
    # Création du BBox valide
    corner = Point2I(corner_x, corner_y)
    bbox = Box2I(corner, Extent2I(cutout_size_pixels, cutout_size_pixels))

    # Extract the cutout from the original exposure
    cutout = exposure.Factory(exposure, bbox, deep=True)

    return cutout, cutout.getWcs(), cutout.getMetadata()

In [ ]:
def RetrieveDRPObject_forTarget(butler,center_coord,datasettype,where_clause,collection,radius_cut=50):
    """
    Find the closest DRP objectto the target_coord 

    parameters:
    - butler
    - the coordinate of the target (center of the cone seach)
    - the datasettype name for the DRP object
    - where_clause : which contrain requirements on the tract and patch numbers
    - cut on angluar separation for the returned for the returned object

    Return
    - object Id with minimum separation , 
    - minimum separation (arcec),
    - the table of DRP objects within the radius_cut
    """

    ra_columns = ['u_ra', 'g_ra', 'r_ra', 'i_ra', 'z_ra', 'y_ra']
    dec_columns = ['u_dec', 'g_dec', 'r_dec', 'i_dec', 'z_dec', 'y_dec']

    
    therefs = butler.registry.queryDatasets(datasettype,  collections=collection, where=where_clause)

    for count,ref in enumerate(therefs):
        the_id = ref.dataId
        the_tract_id = the_id["tract"] 
        print(the_id)
        
        # catalog of rubin objects (a pandas Dataframe) inside the tract
        catalog = butler.get(ref)
        catalog = catalog[catalog["patch"] == patchNbSel] 

        if FLAG_RSPDP1:
            catalog = catalog.to_pandas()
            
       
    
        nobjects = len(catalog)


        # Calcul de la moyenne ligne par ligne, en ignorant les NaN
        catalog['ra'] = catalog[ra_columns].mean(axis=1, skipna=True)
        catalog['dec'] = catalog[dec_columns].mean(axis=1, skipna=True)


        # extract the (ra,dec) coordinates for all te objects of the rubin-catalog
        ra_cat = catalog["ra"].values
        dec_cat = catalog["dec"].values
        # coordinates for all rubin-catalog points
        catalog_coords = SkyCoord(ra=ra_cat*u.deg, dec=dec_cat*u.deg)

        # Angular distance to target
        distances_arcsec = center_coord.separation(catalog_coords).arcsecond

        # add the separation angle to the ctalog
        catalog["sep"] = distances_arcsec


        # closest object from the target
        sepMin = distances_arcsec.min() 
        sepMin_idx = np.where(distances_arcsec == sepMin)[0][0]
    
        closest_obj = catalog[catalog["sep"] <=  sepMin]
                   
        # select a few of these sources to debug the closest candidate
        nearby_obj = catalog[distances_arcsec < radius_cut]
        
        return closest_obj, sepMin, nearby_obj

      

## RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

## Configuration

In [ ]:
FLAG_DUMP_COLLECTIONS = False
FLAG_DUMP_DATASETS = False
FLAG_DUMP_OBJECTSTABLECOLUMNS = False
FLAG_CUT_OBJECTSPRIMARY = False # check this flag may throw the SL
FLAG_CUT_OBJECTSMAG = False
FLAG_CUT_OBJECTSSNR = False

In [ ]:
MAGCUT = 25.0
SNRCUT = 5.0

In [ ]:
all_bands = ["u", "g", "r", "i", "z", "y"]
all_bands_colors = ["blue", "green", "red", "orange", "grey", "purple"]

### Choose instrument

In [ ]:
# instrument = "LSSTCam"

# We focus here on sky fields oberved by LSSTComCam, so we select this camera
instrument = "LSSTComCam"

### Choose options

### For LSSTCam : RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

### For LSSTComCam check here : 
- Check here the collection available : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

In [ ]:
if instrument == "LSSTCam":
    repo = "/repo/embargo"
    instrument = "LSSTCam"
    collection_validation = instrument + "/runs/nightlyValidation"
    # collection_quicklook   = instrument + '/runs/quickLookTesting'
    collection_validation = os.path.join(collection_validation, "20250416/d_2025_04_15/DM-50157")
    date_start = 20250415
    date_selection = 20250416
    where_clause = "instrument = '" + f"{instrument}" + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"
    skymapName = "lsst_cells_v1"
    BANDSEL = "i"

elif instrument == "LSSTComCam":
    repo = "/repo/main"
    


    if FLAG_RSPDP1:
        repo = "dp1"
        collection = "LSSTComCam/DP1"
    else:
        repo = "/repo/main"
        collection = "LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"  # work 2025-05-01
    
    collection_validation = collection
    
    date_start = 20241024
    date_selection = 20241211
    skymapName = "lsst_cells_v1"
    where_clause = "instrument = '" + instrument + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"

    NDET = 9
    TRACTSEL = 5063
    BANDSEL = "i"

In [ ]:
collectionStr = collection_validation.replace("/", "_")

## Access to Butler registry

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection_validation)
registry = butler.registry

## Create a skymap object and Camera

In [ ]:
skymap = butler.get("skyMap", skymap=skymapName, collections=collection_validation)

In [ ]:
camera = butler.get("camera", collections=collection_validation, instrument=instrument)

## Query for collections in Butler

- remove user collections
- remove calibration products

In [ ]:
# mostly setup for LSSTCam
if FLAG_DUMP_COLLECTIONS:
    for _ in sorted(registry.queryCollections(expression=instrument + "/*")):
        if "/calib/" not in _ and "u/" not in _:
            print(_)

## Query for the dataset types in the Butler

- Refer to the Data Product definition Document to know about the definition of datasets
- https://www.lsst.org/about/dm/data-products
- https://lse-163.lsst.io/
- https://docushare.lsst.org/docushare/dsweb/Get/LSE-163

In [ ]:
if FLAG_DUMP_DATASETS:
    for datasetType in registry.queryDatasetTypes():
        if registry.queryDatasets(datasetType, collections=collection_validation).any(
            execute=False, exact=False
        ):
            # Limit search results to the data products
            if (
                ("_config" not in datasetType.name)
                and ("_log" not in datasetType.name)
                and ("_metadata" not in datasetType.name)
                and ("_resource_usage" not in datasetType.name)
                and ("Plot" not in datasetType.name)
                and ("Metric" not in datasetType.name)
                and ("metric" not in datasetType.name)
            ):
                if "object" in datasetType.name or "Obj" in datasetType.name:
                    print(datasetType)
#                if "source" in datasetType.name or "Source" in datasetType.name:
#                    print(datasetType)

## List of Strong Lenses

- article : https://arxiv.org/pdf/1104.0931

In [ ]:
#15422 44 03:32:38.21 –27:56:53.2 
ra1 = "03:32:38.21 hours"
dec1 = "-27:56:53.2 degrees"
tile1 = 44

#34244 94 03:32:06.45 –27:47:28.6 
ra2 = "03:32:06.45 hours"
dec2 = "-27:47:28.6 degrees"
tile2 = 94
# Je ne trouve pas cette tile ==> FindTileForCutoutGEM
tile2 = 32

#40173 35 03:33:19.45 –27:44:50.0 
ra3 = "03:33:19.45 hours"
dec3 = "-27:44:50.0 degrees"
tile3 = 35

#43242 45 03:31:55.35 –27:43:23.5 
ra4 = "03:31:55.35 hours"
dec4 = "-27:43:23.5 degrees"
tile4 = 45

#46446 47 03:31:35.94 –27:41:48.2 
ra5 = "03:31:35.94 hours"
dec5 = "-27:41:48.2 degrees"
tile5 = 47

#12589 03:31:24.89 −27:58:07.0
ra6 = "03:31:24.89 hours"
dec6 = "-27:58:07.0 degrees"
tile6 = 17

#43797 03:31:31.74 −27:43:00.8 
ra7 = "03:31:31.74 hours"
dec7 = "-27:43:00.8 degrees"
tile7 = 47

#28294 03:31:50.54 −27:50:28.4 
ra8 = "03:31:50.54 hours"
dec8 = "-27:50:28.4 degrees"
tile8 = 33

#36857 03:31:53.24 −27:46:18.9
ra9 = "03:31:53.24 hours"
dec9 = "-27:46:18.9 degrees"
tile9 = 38

#36714 03:32:59.78 −27:46:26.4 
ra10 = "03:32:59.78 hours"
dec10 = "-27:46:26.4 degrees"
tile10 = 37


In [ ]:
ra = Angle(ra10)
print(ra.degree)
dec = Angle(dec10)
print(dec.degree)

In [ ]:
lsstcomcam_targets = {}
# high rank
lsstcomcam_targets["ECDFS_G15422"] = {"field_name": "GEMS-15422", "ra": 53.159208333333325, "dec": -27.94811111111111,"tile":tile1}
lsstcomcam_targets["ECDFS_G34244"] = {"field_name": "GEMS-34244", "ra": 53.02687499999999 , "dec": -27.79127777777778,"tile":tile2}
lsstcomcam_targets["ECDFS_G40173"] = {"field_name": "GEMS-40173", "ra": 53.33104166666666 , "dec": -27.747222222222224,"tile":tile3}
lsstcomcam_targets["ECDFS_G43242"] = {"field_name": "GEMS-43242", "ra": 52.980624999999996 , "dec": -27.72319444444444,"tile":tile4}
lsstcomcam_targets["ECDFS_G46446"] = {"field_name": "GEMS-46446", "ra": 52.89975 , "dec": -27.696722222222224,"tile":tile5}

# low rank
lsstcomcam_targets["ECDFS_G12589"] = {"field_name": "GEMS-12589", "ra": 52.85370833333333, "dec": -27.96861111111111,"tile":tile6}
lsstcomcam_targets["ECDFS_G43797"] = {"field_name": "GEMS-43797", "ra": 52.88224999999999, "dec": -27.71688888888889,"tile":tile7}

lsstcomcam_targets["ECDFS_G28294"] = {"field_name": "GEMS-28294", "ra": 52.960583333333325 , "dec": -27.84122222222222,"tile":tile8}
lsstcomcam_targets["ECDFS_G6857"] = {"field_name": "GEMS-6857", "ra": 52.97183333333333 , "dec": -27.771916666666666,"tile":tile9}
lsstcomcam_targets["ECDFS_G36714"] = {"field_name": "GEMS-36714", "ra": 53.249083333333324, "dec": -27.773999999999997,"tile":tile10}


In [ ]:
df = pd.DataFrame(lsstcomcam_targets).T

In [ ]:
df

In [ ]:
# candidates
key = "ECDFS_G15422"
#key = "ECDFS_G34244"
#key = "ECDFS_G40173"
#key= "ECDFS_G43242"
#key= "ECDFS_G46446"

# unknown
#key = "ECDFS_G12589"
#key = "ECDFS_G43797"
#key = "ECDFS_G28294"
#key = "ECDFS_G6857"
#key = "ECDFS_G36714"

the_target = lsstcomcam_targets[key]
target_ra = the_target["ra"]
target_dec = the_target["dec"]
target_name = the_target["field_name"]

target_title = (
    the_target["field_name"] + f" band  {BANDSEL} " + f" (ra,dec) = ({target_ra:.2f},{target_dec:.2f}) "
)
target_point = SpherePoint(target_ra, target_dec, degrees)
target_coord = SkyCoord(ra=target_ra*u.deg, dec=target_dec*u.deg)

## Get list of tracts from the objectTable_tract

In [ ]:
# Find the dimension
if FLAG_RSPDP1:
    print(butler.registry.getDatasetType("object").dimensions)
else:
    print(butler.registry.getDatasetType("objectTable_tract").dimensions)

In [ ]:
if FLAG_RSPDP1:
    datasettype = "object"
else:
    datasettype = "objectTable_tract"
    
therefs = butler.registry.queryDatasets(datasettype, collections=collection_validation)
tractsId_list = np.unique([ref.dataId["tract"] for ref in therefs])
tractsId_list = sorted(tractsId_list)
print(tractsId_list)

## Find the Tract and Patch of the region of interest

- tract in tractNbSel
- patch in patchNbSel

In [ ]:
tract_info = skymap.findTract(target_point)
patch_info = tract_info.findPatch(target_point)
bbox = patch_info.getOuterBBox()

print("Patch bounding box:", bbox)

print("Tract ID :", tract_info.getId())
tractNbSel = tract_info.getId()

print("Patch Index :", patch_info.getIndex(), " , ", patch_info.getSequentialIndex())  # (x, y)
print("Bounding Box", bbox)

patchNbSel = patch_info.getSequentialIndex()

In [ ]:
dataId = {"band": BANDSEL, "tract": tractNbSel, "patch": patchNbSel, "skymap": skymapName}

In [ ]:
full_target_title = target_title + f"(t,p) = ({tractNbSel}, {patchNbSel})"

## The Objects

- Objects are extracted from object detection on deepcoadds
- all bands are included 

### The objects : methods 1

In [ ]:
if FLAG_RSPDP1:
    datasettype = "object"
else:
    datasettype = "objectTable"

In [ ]:
print(butler.registry.getDatasetType(datasettype).dimensions)

In [ ]:
# cannot add a filter on band
where_clause = f"skymap = '{skymapName}' AND tract = {tractNbSel} AND patch = {patchNbSel}"
print(where_clause)

In [ ]:
dataset_refs = list(
    butler.registry.queryDatasets(datasettype, collections=collection_validation, where=where_clause)
)
# Récupère un des refs valides
Nrefs = len(dataset_refs)
print(f"Number of objectTables : {Nrefs}")
ref = dataset_refs[0]
t = butler.get(ref)
Nobj = len(t)
# del t
# gc.collect()
print(f"Total Number of objects {Nobj}")

Oui, dans ce type de objectTable, les colonnes comme g_psfFlux, g_kronFlux, g_cModelFlux, etc., sont des flux calibrés (en nJy ou en unité de calibration interne du pipeline). Pour les convertir en magnitudes AB, tu peux utiliser la formule classique :

In [ ]:
# Utilise ref.datasetType.name pour lister les colonnes disponibles
if FLAG_DUMP_OBJECTSTABLECOLUMNS:
    t_columns = list(butler.get(ref).columns)
    print(t_columns)

In [ ]:
# La constante 31.4 correspond à la conversion AB standard pour un flux exprimé en nanoJanskys (nJy).
# À vérifier selon l’unité exacte utilisée par le pipeline sur ton RSP
# (souvent c’est bien nJy, mais ça peut être autre chose si la calibration a été changée).
# mag = -2.5 * np.log10(flux) + 31.4

## Extract the per-band tables of coordinates

In [ ]:
# Check columns definitions in https://lse-163.lsst.io/
all_radecTable = []

for band in all_bands:
    id_name = "objectId"
    id_parentname = "parentObjectId"
    x_name = f"{band}_centroid_x"
    y_name = f"{band}_centroid_y"
    coord_ra_name = "coord_ra"
    coord_dec_name = "coord_dec"
    ra_name = f"{band}_ra"
    dec_name = f"{band}_dec"
    #decl_name = f"{band}_decl"
    raerr_name = f"{band}_raErr"
    decerr_name = f"{band}_decErr"
    extendedness_name = f"{band}_extendedness"
    blendness_name = f"{band}_blendedness"
    psfflux_name = f"{band}_psfFlux"
    psffluxerr_name = f"{band}_psfFluxErr"
    psfmag_name = f"{band}_psfMag"
    psfmagerr_name = f"{band}_psfMagErr"
    psfflux_free_name = f"{band}_free_psfFlux"
    df = t[
        [
            id_name,
            id_parentname,
            x_name,
            y_name,
            coord_ra_name,
            coord_dec_name,
            ra_name,
            dec_name,
            raerr_name,
            decerr_name,
            #decl_name,
            extendedness_name,
            blendness_name,
            psfflux_name,
            psffluxerr_name,
        ]
    ]

    # select primary objects
    if FLAG_CUT_OBJECTSPRIMARY: 
        #df_sel = (df[df[id_parentname] == 0]).drop([id_parentname], axis=1)
        df_sel = df[df[id_parentname] == 0]
    else:
        df_sel = df

    if FLAG_RSPDP1:
        df_sel = df_sel.to_pandas()

    
    # compute magnitude AB
    df_sel[psfmag_name] = df_sel[psfflux_name].apply(nJy_to_ab_mag)
    df_sel[psfmagerr_name] = nJy_err_to_ab_err(df_sel[psfflux_name], df_sel[psffluxerr_name])

    # drop bad magnitudes
    df_sel = df_sel[df_sel[psfmag_name] != 0.0]
    # drop the fluxes
    # df_sel = df_sel.drop([psfflux_name], axis=1)

    # SNR
    snr = df_sel[psfflux_name] / df_sel[psffluxerr_name]

    if FLAG_CUT_OBJECTSSNR:
        df_sel = df_sel[snr > SNRCUT]

    # select bright objects
    if FLAG_CUT_OBJECTSMAG:
        df_sel = df_sel[df_sel[psfmag_name] < MAGCUT]

    df_sel.reset_index(drop=True, inplace=True)

    # save table in the list
    all_radecTable.append(df_sel)

In [ ]:
for idx, band in enumerate(all_bands):
    df_obj = all_radecTable[idx]
    n = len(df_obj)
    print(f"Number of objects in band {band} : {n}")

In [ ]:
all_radecTable[0]

## Plot Magnitudes and errors

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

for ib, band in enumerate(all_bands):
    color = all_bands_colors[ib]
    df = all_radecTable[ib]
    psfmag_name = f"{band}_psfMag"
    label = f"band {band}"
    df[psfmag_name].hist(ax=ax, bins=50, histtype="step", color=color, label=label)
ax.legend()
ax.set_yscale("log")
ax.set_xlabel("psfMag")
ax.set_title(f"Magnitude of object in (tract,patch) = ({tractNbSel},{patchNbSel})")
ax.set_xlim(12.,40.)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
for ib, band in enumerate(all_bands):
    color = all_bands_colors[ib]
    df = all_radecTable[ib]
    psfmag_name = f"{band}_psfMag"
    psfmagerr_name = f"{band}_psfMagErr"
    label = f"band {band}"
    df.plot.scatter(x=psfmag_name, y=psfmagerr_name, ax=ax, marker=".", s=10, c=color, label=label, alpha=0.5)
    # produce a crash
ax.axhline(0.02,color="k",linestyle=":",label="$5\sigma$ det")
ax.legend()
ax.set_xlabel("psfMag")
ax.set_ylabel("psfMagErr")
ax.set_title(f"Magnitude of Objects in (tract,patch) = ({tractNbSel},{patchNbSel})")
ax.set_aspect("auto")

ax.set_ylim(0.,0.05)
ax.set_xlim(12.,28.)
plt.show()

### Object the methods 2

- the retrieval method is implemented in RetrieveDRPObject_forTarget
- note thos methods does not remove non-primary objects

In [ ]:
closest_obj, sepMin, nearby_obj = RetrieveDRPObject_forTarget(butler,target_coord,datasettype,where_clause,collection=collection_validation)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,4))
nearby_obj["sep"].hist(bins=50,ax=ax,facecolor="b")
closest_obj["sep"].hist(bins=50,range=(0,50),ax=ax,facecolor="r")
ax.set_yscale("log")
ax.set_title("object separation angle")
ax.set_xlabel("separation angle (arcsec)")
plt.show()

In [ ]:
closest_obj["parentObjectId"]

## Select the SL Object among all the objects of the patch

### Study the closest object including by calculating the separation angle
- separation
- extendedness
- blendedness    

In [ ]:
all_df_obj = []
all_df_closeobj = []
for ib,band in enumerate(all_bands):
    df_obj = all_radecTable[ib]

    catalog = SkyCoord(ra=df_obj["coord_ra"].values * u.deg, dec=df_obj["coord_dec"].values * u.deg)
    center = SkyCoord(ra=target_ra * u.deg, dec=target_dec * u.deg)

    # Calcul des distances angulaires
    separations = catalog.separation(center).arcsec
    df_obj["sep"] = separations

    all_df_obj.append(df_obj)
    
    sepMin = separations.min() 
    sepMin_idx = np.where(separations == sepMin)[0][0]
    print(band,sepMin_idx,sepMin)

    all_df_closeobj.append(df_obj.iloc[sepMin_idx])

In [ ]:
fig,axis = plt.subplots(2,3,figsize=(14,8),layout="constrained")
axs = axis.flatten()
for ib,band in enumerate(all_bands):
    df_obj = all_df_obj[ib]
    ax=axs[ib]
    color = all_bands_colors[ib]
    label = f"band {band}"
    df_obj["sep"].hist(bins=50,ax=ax,range=(0,100),color=color, label=label)
    ax.legend()
    ax.set_xlabel("separation angle (arcsec)")
plt.suptitle("Objects separation angle from SL in the different bands")

In [ ]:
all_df_closeobj

## Plot cutout but DO NOT SAVE them in a file (done in SL02)

In [ ]:
all_my_cutouts = []
cutout_pixel_size = 200

for ib,band in enumerate(all_bands):
    iframe = ib + 1

    id_name = "objectId"
   
    id_parentname = "parentObjectId"
    x_name = f"{band}_centroid_x"
    y_name = f"{band}_centroid_y"
    coord_ra_name = "coord_ra"
    coord_dec_name = "coord_dec"
    ra_name = f"{band}_ra"
    dec_name = f"{band}_dec"
    psfmag_name = f"{band}_psfMag" 
    psfmagerr_name = f"{band}_psfMagErr"  

    # object
    df_obj = all_df_closeobj[ib]
    #df_obj[id_name] = df_obj[id_name].astype(int)
    name_obj = int(df_obj[id_name])
    ra_obj = float(df_obj[ra_name])
    dec_obj = float(df_obj[dec_name])
    psfmag_obj = float(df_obj[psfmag_name])
    psfmagerr_obj = float(df_obj[psfmagerr_name])
    title_obj = f"{key} {band} {ra_obj:.2f}, {dec_obj:.2f} m={psfmag_obj:.1f} +/- {psfmagerr_obj:.1f}"
    
    # Extract the cutout around the GEM catalog
    cutout, wcs, metadata = extract_deepCoadd_cutout(
        ra_deg=target_ra,
        dec_deg=target_dec,
        tractInfo=tract_info,
        patchInfo=patch_info,
        band=band,
        butler=butler,
        skymapName=skymapName,
        cutout_size_pixels = cutout_pixel_size,
        collection =  collection_validation
    )

    # Optionnel : nom unique si plusieurs affichages
    display = afwDisplay.Display(frame=iframe)
    display.scale("asinh", "zscale")
    display.setMaskTransparency(90)
    # Affiche le cutout
    display.mtv(cutout.image, title=title_obj)

    
    coord_obj = SpherePoint(ra_obj, dec_obj, degrees)
    pixel_obj = wcs.skyToPixel(coord_obj)  # donne un Point2D (x, y)

    display.dot("x", pixel_obj.getX(), pixel_obj.getY(), size=cutout_pixel_size/10, ctype=all_bands_colors[ib])

    coord_target = SpherePoint(target_ra, target_dec, degrees)
    pixel_target = wcs.skyToPixel(coord_target)  # donne un Point2D (x, y)
    display.dot("+", pixel_target.getX(), pixel_target.getY(), size=cutout_pixel_size/10, ctype=all_bands_colors[ib])
